In [16]:
# Label training data and save it as csv

import pandas as pd
import numpy as np

def makeSportCSV():
    with open('headers_sport.txt', 'r', encoding='utf-8') as f:
        headers = f.readlines()

    with open('headers_corp.csv', 'w', encoding='utf-8') as f:
        for header in headers:
            f.write(header.strip().replace(',','')+',__label__1\n')
            
            
def makeNoSportCSV():
    with open('headers_nosport.txt', 'r', encoding='utf-8') as f:
        headers = f.readlines()

    with open('headers_corp.csv', 'a+', encoding='utf-8') as f:
        for header in headers:
            f.write(header.strip().replace(',','')+',__label__2\n')
            
# makeSportCSV()
# makeNoSportCSV()

In [ ]:
# Replace nltk stemmer with Polish stemmer version from pystempel repository (https://github.com/dzieciou/pystempel)

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
from stempel import StempelStemmer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

# iloc[:3] - reduce number of headers for testing purpose 
 
Corpus = pd.read_csv(r"headers_corp.csv",encoding='utf-8').iloc[:6]

In [ ]:
# Preprocess text and format to lower case.

def preprocess():
    Corpus['text'].dropna(inplace=True)
    Corpus['text'] = [entry.lower() for entry in Corpus['text']]

    # Tokenize text
    Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]

    # Remove polish stopwords and non alphabetical
    # Stem rest of the words

    for index,entry in enumerate(Corpus['text']):

        Final_words = []
        stemmer = StempelStemmer.default()

        for word in entry:
            # it's necessary to add polish stopwords to nltk_data/corpora/stopwords
            if word not in stopwords.words('polish') and word.isalpha():
                word_Final = stemmer.stem(word)
                Final_words.append(word_Final)

        # The final preprocessed set of words for each iteration will be stored in 'text_final'
        Corpus.loc[index,'text_final'] = str(Final_words)
        
        
preprocess()

In [ ]:
# Split data to train dataset and validation dataset 

Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

# Transform datasets to vectors with TFIDF vectorizer

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Train_X)
print()
print(Test_X_Tfidf)

In [ ]:
# Fit the training dataset on the Naive Bayes classifier

model = naive_bayes.MultinomialNB()
model.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_NB = model.predict(Test_X_Tfidf)

# # Use accuracy_score function to get the accuracy
# print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)